In [1]:
import pandas as pd
import numpy as np
import csv
import os

In [2]:
import pandas as pd
import numpy as np
import csv
import os


# read flash.dat to a list of lists
datContent = [i.strip().split() for i in open(r'hetrec2011-lastfm-2k\user_taggedartists-timestamps.dat').readlines()]
df=pd.DataFrame(datContent[1:],columns=datContent[0])
df.to_csv(r'hetrec2011-lastfm-2k\user_taggedartists-timestamps.csv')

In [3]:
df=pd.DataFrame(datContent[1:],columns=datContent[0])

In [4]:

def walkFile(file):
    filelist=[]
    for root, dirs, files in os.walk(file):
        for f in files:
            filelist.append(os.path.join(f))

        return filelist
filelist=walkFile(r"hetrec2011-lastfm-2k")
filelist

['artists.dat',
 'readme.txt',
 'tags.dat',
 'user_artists.dat',
 'user_friends.dat',
 'user_taggedartists-timestamps.csv',
 'user_taggedartists-timestamps.dat',
 'user_taggedartists.dat']

In [ ]:
#convert to csv
datContent = [i.strip().split('	') for i in open(r'hetrec2011-lastfm-2k\artists.dat','r', encoding='mac_roman').readlines()]
df=pd.DataFrame(datContent[1:],columns=datContent[0])
df.to_csv(r'hetrec2011-lastfm-2k\artists.csv')
print(r'hetrec2011-lastfm-2k\artists.csv')
for i in filelist[2:]:
    datContent = [i.strip().split('	') for i in open(r'hetrec2011-lastfm-2k\\'+i, encoding='mac_roman').readlines()]
    df=pd.DataFrame(datContent[1:],columns=datContent[0])
    i=i.strip('.dat')+'.csv'
    df.to_csv(r'hetrec2011-lastfm-2k\\'+i)
    print(r'hetrec2011-lastfm-2k\\'+i)

In [6]:
artists=pd.read_csv(r'hetrec2011-lastfm-2k\artists.csv')
artists.drop(['Unnamed: 0'],axis=1, inplace=True)
tags=pd.read_csv(r'hetrec2011-lastfm-2k\gs.csv')
tags.drop(['Unnamed: 0'],axis=1, inplace=True)
user_taggedartists_timestamps=pd.read_csv(r'hetrec2011-lastfm-2k\user_taggedartists-timestamps.csv')
user_taggedartists_timestamps.drop(['Unnamed: 0'],axis=1, inplace=True)
user_taggedartists_timestamps=user_taggedartists_timestamps.sort_values(by=['timestamp'])
user_taggedartists=pd.read_csv(r'hetrec2011-lastfm-2k\user_taggedartists.csv')
user_taggedartists.drop(['Unnamed: 0'],axis=1, inplace=True)

In [7]:
user_taggedartists

,userID,artistID,tagID,day,month,year
0,2,52,13,1,4,2009
1,2,52,15,1,4,2009
2,2,52,18,1,4,2009
3,2,52,21,1,4,2009
4,2,52,41,1,4,2009
...,...,...,...,...,...,...
186474,2100,16437,4,1,7,2010
186475,2100,16437,292,1,5,2010
186476,2100,16437,2087,1,7,2010
186477,2100,16437,2801,1,5,2010


In [11]:
#100 most frequent users
frequentUser=np.array(user_taggedartists.userID.value_counts().index[:500])
#10 most frequent items
frequentItem=list(user_taggedartists.artistID.value_counts().index[:6000])
frequentUserData=user_taggedartists_timestamps[(user_taggedartists_timestamps['userID'].isin(frequentUser))].sort_values(by=['timestamp'])
freqData=user_taggedartists_timestamps[(user_taggedartists_timestamps['userID'].isin(frequentUser))&(user_taggedartists_timestamps['artistID'].isin(frequentItem))].sort_values(by=['timestamp'])
userFlow=list(freqData.userID)
trueResultFlow=list(freqData.artistID)
len(freqData)

146242

In [13]:
np.save(r'hetrec2011-lastfm-2k\newData.npy',newData)
newData=np.load(r'hetrec2011-lastfm-2k\newData.npy')

In [19]:
newDataDict={}
for i in range(len(newData)):
    newDataDict[tags.tagID[i]]=newData[i]

frequentItemDict={}
for i in frequentItem:
    temp=np.zeros(10)
    for j in list(user_taggedartists_timestamps[user_taggedartists_timestamps.artistID==i].tagID):
        if j<len(newData):
            temp+=newDataDict[j]
    if np.linalg.norm(temp, ord=2)==0.:
        print(i)
    frequentItemDict[i]=temp/np.linalg.norm(temp, ord=2)

In [21]:
clf1 = KMeans(n_clusters=40)
s1 = clf1.fit(list(frequentItemDict.values()))

frequentItemCluster={}
for i in range(len(frequentItem)):
    frequentItemCluster[frequentItem[i]]=clf1.labels_[i]
frequentItemVec={}
for i in range(40):
    frequentItemVec[i]=np.zeros(10)
for i in frequentItem:
    frequentItemVec[frequentItemCluster[i]]+=frequentItemDict[i]
for i in range(40):
    frequentItemVec[i]=frequentItemVec[i]/np.linalg.norm(frequentItemVec[i], ord=2)

stage1Data=user_taggedartists_timestamps
#stage2Data=user_taggedartists_timestamps[50000:100000]
#stage3Data=user_taggedartists_timestamps[100000:150000]
#stage4Data=user_taggedartists_timestamps[150000:]
frequentUserDict1={}
#frequentUserDict2={}
#frequentUserDict3={}
#frequentUserDict4={}
for i in frequentUser:
    temp=np.zeros(10)
    for j in list(stage1Data[stage1Data.userID==i].tagID):
        if j<len(newData):
            temp+=newDataDict[j]
    if np.linalg.norm(temp, ord=2)==0:
        frequentUserDict1[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)
    else:
        frequentUserDict1[i]=temp/np.linalg.norm(temp, ord=2)
        
"""
for i in frequentUser:
    temp=np.zeros(10)
    for j in list(stage2Data[stage2Data.userID==i].id):
        temp+=newDataDict[j]
    if np.linalg.norm(temp, ord=2)==0:
        frequentUserDict2[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)
    else:
        frequentUserDict2[i]=temp/np.linalg.norm(temp, ord=2)
for i in frequentUser:
    temp=np.zeros(10)
    for j in list(stage3Data[stage3Data.userID==i].id):
        temp+=newDataDict[j]
    if np.linalg.norm(temp, ord=2)==0:
        frequentUserDict3[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)
    else:
        frequentUserDict3[i]=temp/np.linalg.norm(temp, ord=2)
for i in frequentUser:
    temp=np.zeros(10)
    for j in list(stage4Data[stage4Data.userID==i].id):
        temp+=newDataDict[j]
    if np.linalg.norm(temp, ord=2)==0:
        frequentUserDict4[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)
    else:
        frequentUserDict4[i]=temp/np.linalg.norm(temp, ord=2)

freqUserFeatureDict={}
for i in frequentUser:
    freqUserFeatureDict[i]=[]
    for j in range(len(freqData)//4):
        freqUserFeatureDict[i].append(frequentUserDict1[i])
    for j in range(len(freqData)//4):
        freqUserFeatureDict[i].append(frequentUserDict2[i])
    for j in range(len(freqData)//4):
        freqUserFeatureDict[i].append(frequentUserDict3[i])
    for j in range(len(freqData)-3*len(freqData)//4):
        freqUserFeatureDict[i].append(frequentUserDict4[i])
"""


'\nfor i in frequentUser:\n    temp=np.zeros(10)\n    for j in list(stage2Data[stage2Data.userID==i].id):\n        temp+=newDataDict[j]\n    if np.linalg.norm(temp, ord=2)==0:\n        frequentUserDict2[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)\n    else:\n        frequentUserDict2[i]=temp/np.linalg.norm(temp, ord=2)\nfor i in frequentUser:\n    temp=np.zeros(10)\n    for j in list(stage3Data[stage3Data.userID==i].id):\n        temp+=newDataDict[j]\n    if np.linalg.norm(temp, ord=2)==0:\n        frequentUserDict3[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)\n    else:\n        frequentUserDict3[i]=temp/np.linalg.norm(temp, ord=2)\nfor i in frequentUser:\n    temp=np.zeros(10)\n    for j in list(stage4Data[stage4Data.userID==i].id):\n        temp+=newDataDict[j]\n    if np.linalg.norm(temp, ord=2)==0:\n        frequentUserDict4[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)\n    else:\n        frequentUserDict4[i]=temp/np.linalg.norm(temp, ord=2)\n\nfreqUserFeatureDict=

In [22]:
np.save(r'hetrec2011-lastfm-2k\frequentUserDict300_del.npy',frequentUserDict1)
np.save(r'hetrec2011-lastfm-2k\frequentItemVec.npy',frequentItemVec)

In [23]:
indexflow=[]
for i in range(len(user_taggedartists_timestamps)):
    if i%500==0:
        print(i)
    if user_taggedartists_timestamps.iloc[i].artistID in list(frequentItemDict.keys()):
        indexflow.append(i)
artistflow=user_taggedartists_timestamps.iloc[indexflow].artistID.values
userflow=user_taggedartists_timestamps.iloc[indexflow].userID.values
artistflow=[frequentItemCluster[i] for i in artistflow]
artistflow

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000
84500
8500

[7,
 37,
 38,
 26,
 24,
 24,
 24,
 34,
 29,
 29,
 28,
 29,
 25,
 25,
 5,
 25,
 3,
 3,
 3,
 6,
 6,
 6,
 6,
 6,
 1,
 1,
 3,
 35,
 25,
 25,
 18,
 35,
 35,
 35,
 35,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 23,
 23,
 23,
 23,
 24,
 24,
 24,
 25,
 25,
 25,
 23,
 37,
 35,
 37,
 29,
 29,
 5,
 37,
 6,
 6,
 24,
 24,
 24,
 24,
 26,
 29,
 29,
 37,
 20,
 37,
 1,
 29,
 20,
 24,
 36,
 26,
 29,
 37,
 26,
 6,
 29,
 37,
 6,
 6,
 37,
 37,
 37,
 37,
 37,
 35,
 6,
 35,
 35,
 35,
 37,
 29,
 29,
 29,
 29,
 5,
 5,
 5,
 35,
 37,
 6,
 6,
 1,
 1,
 25,
 37,
 1,
 1,
 24,
 24,
 1,
 29,
 1,
 5,
 24,
 25,
 24,
 24,
 20,
 35,
 35,
 29,
 20,
 5,
 24,
 23,
 24,
 37,
 37,
 24,
 24,
 2,
 2,
 2,
 2,
 25,
 1,
 1,
 24,
 24,
 37,
 10,
 12,
 25,
 25,
 37,
 1,
 1,
 29,
 24,
 20,
 20,
 37,
 29,
 35,
 35,
 35,
 26,
 26,
 5,
 5,
 5,
 5,
 35,
 35,
 6,
 6,
 6,
 6,
 6,
 26,
 26,
 35,
 29,
 35,
 26,
 6,
 37,
 18,
 37,
 1,
 6,
 6,
 6,
 6,
 6,
 24,
 24,
 29,
 29,
 24,
 24,
 24,
 24,
 29,
 29,
 39,
 39,
 39,
 39,
 37,
 37,
 35,
 1,
 35,
 29,

In [25]:
np.save(r'hetrec2011-lastfm-2k\userflow.npy',userflow)
np.save(r'hetrec2011-lastfm-2k\artistflow.npy',artistflow)
np.save(r'hetrec2011-lastfm-2k\frequentItemVec.npy',np.array(list(frequentItemVec.values())))

In [26]:
userflowdict={}
userflowlist=list(set(userflow))
for i in range(len(set(userflow))):
    userflowdict[userflowlist[i]]=i
userclick=[[] for i in range(len(set(userflow)))]
for i in range(len(userflow)):
    userclick[userflowdict[userflow[i]]].append(artistflow[i])
userflowRenew=[]
for i in userflow:
    userflowRenew.append(userflowdict[i])

In [1]:
np.save(r'hetrec2011-lastfm-2k\userclick.npy',userclick)
np.save(r'hetrec2011-lastfm-2k\userflow.npy',userflowRenew)

NameError: name 'np' is not defined